----

# Analyzing borrowers’ risk of defaulting

The purpose of this project is to analyze the factors that determine the credit worthiness of future potential customers of a bank. As we already have data on the credit worthiness of existing customers, we will use these to test varying hypotheses that will form a criteria for future loan approvals moving forward.

## Hypotheses:
<br>
a) Default rates are affected by an individual's number of children.
<br>b) Default rates are affected by an individual's family status.
<br>c) Other factors play a role in the default rate of an individual: (i) Income bracket, (ii) Purpose of loan

In [1]:
# Importing necessary Libararies

import pandas as pd

In [2]:
# Loading Data
try: 
    df = pd.read_csv('credit_scoring_eng.csv')
except: 
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan



In [3]:
# Finding our DataFrame shape:

df.shape

(21525, 12)

In [4]:
# Getting a brief overview of our data: 

display(df.head(10))
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,secondary education,1,married,0,F,employee,1,17517.812,to become educated
21516,0,-914.391429,42,bachelor's degree,0,married,0,F,business,0,51649.244,purchase of my own house
21517,0,-404.679035,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.529,buying my own car
21518,0,373995.710800,59,SECONDARY EDUCATION,1,married,0,F,retiree,0,24618.344,purchase of a car
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404100,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


**Potential issues with our data:** 
1. The 'purpose' column has unique duplicates of the same categorical value. Ie: 'to have a wedding' & 'having a wedding'. These will need to replaced.
2. There are negative values in the 'children' column, which does not make any sense.
3. The 'days_employed' column has some extremely high values, which does not make sense.
4. Difference in letter-casing in our 'education' column.

In [5]:
# Getting a summary of our DataFrame:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



Missing values are apparent in columns 'days_employed' and 'total_income'. A possible explanation for this would be that certain values were accidentally ommitted due to human/machine error or they were done so on purpose due to a dependent conditional variable. The null values seem to tally however, so perhaps there is a pattern to investigate.

In [6]:
# Filtering df by missing values of 'days_employed' column:

display(df[df['days_employed'].isna()].head()) 
df[df['days_employed'].isna()].tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate
21510,2,NaN,28,secondary education,1,married,0,F,employee,0,NaN,car purchase


Based off of the DataFrame comprising missing values in our 'days_employed' column, the head and tail  suggests that our assumption is right. Each missing values in our filtered 'days_employed' column correlate symmetrically with a missing value in the 'total_income' column. 
<br>
<br>HOWEVER, we cannot assume this is consistent across thousands of lines, and that we may also have one column present while the other is missing. This needs to be confirmed:


In [7]:
# Applying two conditions to prove our assumption that missing values in both columns are symmetrical:
df_missing = df[(df['days_employed'].isna()) & (df['total_income'].isna())]

# Getting the total null count of our filtered DataFrame
print(len(df_missing)) 

# Getting the total null count of the original DataFrame
len(df) - df['total_income'].count()

2174


2174

**Intermediate conclusion**
<br> The number of null values of each column in our filtered table tally with our original DataFrame. We can conclude that individuals with missing values have similar characteristics in that either 
<br>
(a) their data was omitted from the dataset due to a dependent condition or 
<br>
(b) it was purely an error. We should be able to safely classify their data together.
<br>
<br>Let's assume missing values are dependent on 'income_source', and that our entries with missing values are individuals who do not have income.
<br>



**Conditional Hypothesis: Individuals with missing values are those that have stopped earning or do not have an income stream.**


In [8]:
# Finding the percentage of missing values across the entire DataFrame:

df.isna().mean() * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

A 10% missing share is a big enough number that we should worry about misrepresentation if we were to just drop those particular rows. We have to fill our missing values in instead, but we first need to determine if they are random or correlated to another column. 

In [9]:
# Finding unique indentifiers of 'income_type'

df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

There are 3 possible variables under column 'income_type' that potentially fit the condition of {*NO INCOME STREAM*}: 

(a) retiree
<br>
(b) unemployed 
<br>
(c) student.
<br>

In [10]:
# Getting missing value count for each unique 'income_type' indentifier:

display(df_missing['income_type'].value_counts())

# Getting the percentage of missing values of 'retirees' across the entire Series.

(df_missing['income_type']=='retiree').mean() * 100

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

18.997240110395584

*Condition of {NO INCOME STREAM} = 'retiree'*
<br>
While we found a variable that fits our condition, it only amounts to 19% of our entire missing value data. The biggest representation of our missing value data seems to be the 'employee' variable followed by 'business', both of which do not fit our condition.  
<br> Moreover, other variables of our {*NO INCOME STREAM*} condition are not present. There does not seem to be a dependency pattern here. 

In [11]:
# Finding out the distribution of 'retiree' values in our original dataset:

df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

There seem to be other 'retiree' variables that did not show up in our table of missing values. We can conclude that there is no dependency pattern on 'income_source'.

*Conditional hypothesis is rejected.*


**Possible reasons for missing values in data**
There's a high chance that missing values were present due to malfunctions/bugs in the data pipeline. It's highly unlikely the errors were man-made, as they were consistent across both columns, and the values in the rest of the columns were filled out. This is highly likely a randomized error, but we still have to confirm this.

**Missing values are random and hence, accidental.**

In [12]:
# Checking the distribution of both the original dataset and filtered dataset with missing values:
display(df_missing.describe())
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949900,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423626,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400500,75.000000,4.000000,4.000000,1.000000,362496.645000


**Intermediate conclusion**

Most aggregate indentifiers are similar between both datasets, apart from the 25th Percentile Median of column 'education_id'. Our means are the most telling, as are our standard deviations in either dataset. They are within range of each other across all columns and this serves the notion that our filtered dataset is a substantially representative subset of our entire dataset.

Missing values are randomized.


To be 100% sure, we will calculate the median for the entire columns of 'dob_years' and 'education' to account for any outliers that may skew our distribution.

In [13]:
# Age Median

print('Age median of filtered dataset: ', df_missing['dob_years'].median())
print('Age median of dataset: ', df['dob_years'].median())

Age median of filtered dataset:  43.0
Age median of dataset:  42.0


In [14]:
# Education Median

print('Education median of filtered dataset: ', df_missing['education_id'].median())
print('Education median of dataset: ', df['education_id'].median())

Education median of filtered dataset:  1.0
Education median of dataset:  1.0


**Intermediate conclusion**

*Hypothesis is accepted: Our missing values seem to be purely accidental and randomized.*

**Conclusions**

There was randomized theme to our missing values, as the distribution of aggregate data on our filtered dataset are symmetrical with our original dataset.
<br>
<br>
Dealing with our missing values:
Our values in the 'dob_years' column need to be grouped into *(1) age ranges*, so we can find the mean/median salary and work experience by age groups, corresponding with their *(2) education level*.
<br>
<br>
Data Transformation:
<br>
a) 'days_employed' and 'children' values need to be converted to positive floats
<br>
b) 'education' values need to have standardized lower casing
<br>
c) 'purpose' values have numerous unique duplicates and need to be classified under a single unique variable


## Data transformation

### Education Column

In [15]:
# Checking for spelling errors of the 'education' column by finding its unique values

display(df['education'].unique())
df['education_id'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

array([0, 1, 2, 3, 4], dtype=int64)

There are multiple unique duplicates of the same education id, all with different spellings.

In [16]:
# Finding the unique values for each 'education_id':

df.groupby('education_id')['education'].unique()

education_id
0    [bachelor's degree, BACHELOR'S DEGREE, Bachelo...
1    [secondary education, Secondary Education, SEC...
2           [some college, SOME COLLEGE, Some College]
3    [primary education, PRIMARY EDUCATION, Primary...
4    [Graduate Degree, GRADUATE DEGREE, graduate de...
Name: education, dtype: object

Apart from lower/upper case formatting issues, we don't seem to have any other spelling errors present.

In [17]:
# Formatting all values of our 'education' column into lower case: 

df['education'] = df['education'].str.lower()

In [18]:
# Making sure our education registers are fixed:

df.groupby('education_id')['education'].unique()

education_id
0      [bachelor's degree]
1    [secondary education]
2           [some college]
3      [primary education]
4        [graduate degree]
Name: education, dtype: object

### Children Column

In [19]:
# Finding the distribution of unique values in the `children` column:

df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Strange values are apparent: 20, -1

In [20]:
# Finding the percentage share of unique value (-1):

display((df['children'] == -1).mean() * 100)
display((df['children'] == 20).mean() * 100)

((df['children'] == -1).mean() * 100) + ((df['children'] == 20).mean() * 100)

0.2183507549361208

0.3530778164924506

0.5714285714285714

There appears to be a unique value of 20 and -1; it is illogical to have -1 children in a family, nor is it 'likely' someone would have 20 children. This could have been due to a typo error. Since percentage share of 20 and -1 children against our entire dataset is extremely negligible at 0.57%, it is best to drop these rows altogether as the value could mean anything and we do not want any misrepresentation in a column that is important to our credit rating objectives.  

In [21]:
# Dropping rows that == -1:

df.drop(df[df['children'] == -1].index, inplace = True)


In [22]:
# Dropping rows that == 20:

df.drop(df[df['children'] == 20].index, inplace = True)


In [23]:
# Checking our unique values for the 'children' column again:

df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Days Employed Column

In [24]:
## Find problematic data in `days_employed`, if they exist, and calculate the percentage: 

display(df['days_employed'].value_counts())

-8437.673028      1
-3963.590317      1
 356425.151400    1
-3112.788664      1
-5135.928528      1
                 ..
 353207.809200    1
-1083.159959      1
 390769.069000    1
 341277.436000    1
-1984.507589      1
Name: days_employed, Length: 19240, dtype: int64

From our value_counts method, we've found two obvious errors:
<br>
<br>
a) negative days of employment
<br>
b) an extremely overestimated days of employment

In [25]:
# Calculating the percentage share of negative, overestimated and null values of our dataset:

negative_values = df[df['days_employed'] < 0]['days_employed'].count()
display(negative_values)
display(negative_values / len(df['days_employed']))

overestimated_values = df[df['days_employed'] > 0]['days_employed'].count()
display(overestimated_values)
display(overestimated_values / len(df['days_employed']))

null_values =  df['days_employed'].isna().sum()
display(null_values)
display(null_values / len(df['days_employed']))

15809

0.7386692832445566

3431

0.16031212036258294

2162

0.10101859639286048

In [26]:
# Confirming if counts tally with total entries in the dataset:

(21478 - (negative_values + null_values)) == overestimated_values 

False

**FINDINGS:**
<br>
1) The majority of 'days_employed' values (74%) are a negative value, which suggests they were calculated by subtracting an individual's present age (in years) from the year they started working, instead of the other way around. This could be a flaw in the code that was used to automatically calculate the 'days_employed' column. 
<br>
<br>
2) There appears to be largely overestimated number of employed days for some/all individuals that possess a positive non-null value. For instance, 337451.389600 days / 365 days = 924.52 years. This is illogical! 


In [27]:
# Checking if all positive values are consistently overestimated:

df[df['days_employed'] > 0]['days_employed'].sort_values(ascending=True)

20444    328728.7206
9328     328734.9240
17782    328771.3414
14783    328795.7267
7229     328827.3457
            ...     
7794     401663.8500
2156     401674.4666
7664     401675.0934
10006    401715.8117
6954     401755.4005
Name: days_employed, Length: 3431, dtype: float64

The lowest positive float of our column is indeed overestimated by the very least a factor of 100, so we can group these values as such. 
<br>As it is impossible find out the actual values without looking at the data pipeline, we need to find out if we can remove them completely by looking for correlations between our 'days_employed' column and other columns (quite possibly 'income_type')

In [28]:
# Testing if there is any correlation with our overestimated values and the rest of our values with 'income_type'.

overestimated_filt = df[df['days_employed'] > 0]  # FILTERED TABLE for our overestimated data
negative_filt = df[df['days_employed'] < 0]  # FILTERED TABLE for our negative values


display(overestimated_filt.groupby('income_type')['days_employed'].count())
display(negative_filt.groupby('income_type')['days_employed'].count())


income_type
retiree       3429
unemployed       2
Name: days_employed, dtype: int64

income_type
business                       4550
civil servant                  1307
employee                       9949
entrepreneur                      1
paternity / maternity leave       1
student                           1
Name: days_employed, dtype: int64

**Intermediate Conclusion**

It appears that our non-null data have been classified by those who currently have a stable income stream and those who do not (retirees and unemployed). 
<br>
Understanding that our case study involves assessing the credit rating based off of an individual's family status and no. of children,  we can omit data that pertains to the group that does not have means to stable income for various reasons:
<br>
<br>
a) they are possibly at the end of their loan repayments or have already finished paying it back
<br>b) highly likely will not be borrowing further, and hence have no need for a credit rating moving forward
<br>c) their children (if any) are likely to have grown up and are no longer dependent on them
<br>d) if not any of the above, purely for the fact that they do not have a guarantee of income stability, assuming they do not go into business. 


In [29]:
# Removing all our non-null rows with overestimated 'days_employed' values:

(df.drop(df[df['days_employed'] > 0].index, inplace = True))

In [30]:
# Rechecking our data:

df[df['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [31]:
# Fixing negative values in the 'days_employed' col:

# by converting negative values to a positive float, that is their absolute value:

df['days_employed'] = df['days_employed'].abs()

In [32]:
# Checking if the fix was executed:

display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17971 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          17971 non-null  int64  
 1   days_employed     15809 non-null  float64
 2   dob_years         17971 non-null  int64  
 3   education         17971 non-null  object 
 4   education_id      17971 non-null  int64  
 5   family_status     17971 non-null  object 
 6   family_status_id  17971 non-null  int64  
 7   gender            17971 non-null  object 
 8   income_type       17971 non-null  object 
 9   debt              17971 non-null  int64  
 10  total_income      15809 non-null  float64
 11  purpose           17971 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house


### Dob Years (Age) Column

In [33]:
# Checking the `dob_years` col:

display(df['dob_years'].value_counts().sort_index()) 
df[df['dob_years'] == 0]['dob_years'].count()/len(df) #finding the percentage share of strange values

0      83
19     14
20     51
21    110
22    182
23    252
24    263
25    356
26    404
27    487
28    501
29    543
30    536
31    555
32    503
33    575
34    594
35    613
36    550
37    526
38    587
39    568
40    596
41    597
42    583
43    501
44    533
45    483
46    456
47    467
48    517
49    476
50    449
51    373
52    388
53    352
54    332
55    280
56    299
57    245
58    253
59    187
60    133
61    140
62    117
63     77
64     86
65     58
66     44
67     35
68     19
69     11
70     11
71     10
72      5
73      2
74      2
75      1
Name: dob_years, dtype: int64

0.004618552111735574

'0' is an illogical age, and since these values represent 0.004% share of our subset, we can drop them.

In [34]:
# dropping the rows whereby 'dob_years' == 0

df.drop(df[df['dob_years'] == 0].index, inplace = True)

In [35]:
# Checking if our 'dob_years' col is fixed

df['dob_years'].value_counts().sort_index()

19     14
20     51
21    110
22    182
23    252
24    263
25    356
26    404
27    487
28    501
29    543
30    536
31    555
32    503
33    575
34    594
35    613
36    550
37    526
38    587
39    568
40    596
41    597
42    583
43    501
44    533
45    483
46    456
47    467
48    517
49    476
50    449
51    373
52    388
53    352
54    332
55    280
56    299
57    245
58    253
59    187
60    133
61    140
62    117
63     77
64     86
65     58
66     44
67     35
68     19
69     11
70     11
71     10
72      5
73      2
74      2
75      1
Name: dob_years, dtype: int64

### Family Status Column

In [36]:
# Checking 'family_status' col:

df['family_status'].unique()

array(['married', 'civil partnership', 'divorced', 'unmarried',
       'widow / widower'], dtype=object)

The 'family_status' col does not seem to have any strange categories.



### Gender Column

In [37]:
# Checking the 'gender' col:

df['gender'].value_counts()

F      11304
M       6583
XNA        1
Name: gender, dtype: int64

The strange categorical value of 'XNA' only makes up for 1 entry, so we will just remove it completely.

In [38]:
# Dropping rows of 'gender' values == 'XNA', as they represent an extremely small subset of our dataset

df.drop(df[df['gender'] == 'XNA'].index, inplace = True)

In [39]:
# Checking if our 'gender' col is fixed:

df['gender'].value_counts()

F    11304
M     6583
Name: gender, dtype: int64

### Income Type Column

In [40]:
# Checking the 'income_type' col:

df['income_type'].value_counts()

employee                       10996
business                        5033
civil servant                   1447
retiree                          407
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

The 'income_type' col does not seem to have any strange categorical values.

In [41]:
# Checking for duplicates

duplicated_df = df[df.duplicated()]
duplicated_df['days_employed'].unique()

array([nan])

It is impossible to find obvious complete duplicates without a unique customer id for each entry made. We could possibly use 'days_employed' as our subset, as the nature of these values are unique in and of itself. However, its values are completely missing. 
<br>
Hence, we cannot drop these rows because there's nothing to uniquely diffrentiate one entry from another.

In [42]:
# Looking at our finalized dataset size after data transformation: 

df.shape

(17887, 12)

### Data Transformation Conclusion -  
<br>
We originally started with 21525 entries in our dataset, and have managed to drop 3638 entries that equates to 17% of our DataFrame. We got here by eliminating a subset of data of that would have skewed any kind of mean/median values we attained for  the purpose of filling in missing values. Furthermore, this subset belongs to a group of individuals who are less important to our main overall goal of credit worthiness in terms of family status and no. of children. We still have a good amount of data that are representative of our objectives to work with.

# Working with missing values

For our missing values in columns 'days_employed' and 'total_income', we can attribute them to an individual's age, education level and gender. This will approximate how long they've been working throughout their lifetime, how much longer they have in the workforce to contribute to loan repayments, and the type of pay they would expect to be receive in the job market at certain junctures in their age cycle. To do this we need:
<br>
a) groups of age ranges
<br>
b) mean/median based off of age groups, education level and gender

In [43]:
# Getting our education dictionary:

education_dict = {
    0: "bachelor's degree",
    1: "secondary education",
    2: "some college",
    3: "primary education",
    4: "graduate degree"
}

### Restoring missing values in `total_income` based off of `age_group`.

#### Reasonings and Assumptions
<br>
Our missing values in columns 'days_employed' and 'total_income' can be estimated with the mean/median of grouped age ranges, gender and education level respectively. This is due to the calculated assumption that:
<br>
<br>
a) Education Level - All other factors held constant, individual who starts work immediately after finishing primary school is more likely to be further along his/her career path, having ammassed more employment experience, compared to one who has just graduated from a graduate degree. 
<br>b) Age group - Similarly, someone who is 70 years old is likely to have ammassed more working experience and a higher pay than someone who is 30 years old.
<br>c) Gender - Based off of socioeconomic factors, there might be an income disparity between males and females we have to account for. Furthermore, we also have to factor in the likelihood female individuals might take a career break for childbirth.

In [44]:
# Getting age ranges to determine our age groupings:

df['dob_years'].value_counts().sort_index()   

19     14
20     51
21    110
22    182
23    252
24    262
25    356
26    404
27    487
28    501
29    543
30    536
31    555
32    503
33    575
34    594
35    613
36    550
37    526
38    587
39    568
40    596
41    597
42    583
43    501
44    533
45    483
46    456
47    467
48    517
49    476
50    449
51    373
52    388
53    352
54    332
55    280
56    299
57    245
58    253
59    187
60    133
61    140
62    117
63     77
64     86
65     58
66     44
67     35
68     19
69     11
70     11
71     10
72      5
73      2
74      2
75      1
Name: dob_years, dtype: int64

In [45]:
# Function to calculate the age categories:

def assign_age_group(age):
    if age <= 19:
        return '10s'
    if age <= 29:
        return '20s'
    if age <= 39:
        return '30s'
    if age <= 49:
        return '40s'
    if age <= 59:
        return '50s'
    if age <= 69:
        return '60s'
    elif age <= 79:
        return '70s'


In [46]:
# Testing if the function works: 

display(assign_age_group(39))
display(assign_age_group(72))
assign_age_group(19)

'30s'

'70s'

'10s'

In [47]:
# Creating new column = 'age_group' based on age_group function:

df['age_group'] = df['dob_years'].apply(assign_age_group)

In [48]:
# Create a table WITHOUT missing values to compute our means and medians for 'total_income':

non_null_df = df.dropna()

In [49]:
#Getting our income mean values:

display("INCOME MEAN")
income_mean = non_null_df.pivot_table(index=['age_group', 'education', 'gender'], values='total_income', aggfunc='mean')
income_mean

'INCOME MEAN'

total_income
age_group education           gender              
10s       secondary education F       16409.592667
                              M       14817.591000
          some college        F       19563.199250
                              M       14575.717000
20s       bachelor's degree   F       27113.420437
                              M       33753.467111
          primary education   F       23193.671000
                              M       29445.893944
          secondary education F       20607.714651
                              M       26643.539464
          some college        F       22252.881988
                              M       30075.849101
30s       bachelor's degree   F       30827.044174
                              M       40738.531160
          graduate degree     F       17822.757000
                              M       18551.846000
          primary education   F       21351.115667
                              M       22599.818950
          secondary education F       23096.506566
                              M       29534.213058
          some college        F       29493.266488
                              M       35848.935671
40s       bachelor's degree   F       32113.464446
                              M       43752.030344
          graduate degree     M       31771.321000
          primary education   F       21855.720048
                              M       25731.594440
          secondary education F       23776.018462
                              M       30713.246399
          some college        F       32661.378773
                              M       37201.683081
50s       bachelor's degree   F       34972.059017
                              M       40803.490246
          graduate degree     M       42945.794000
          primary education   F       15963.277357
                              M       21924.730000
          secondary education F       24100.635847
                              M       27887.339902
          some college        F       28187.453136
                              M       35454.835143
60s       bachelor's degree   F       31189.781975
                              M       39987.492875
          primary education   F       25508.640000
                              M       19490.938500
          secondary education F       26991.866856
                              M       26075.780964
          some college        F       33081.610000
                              M       28373.852000
70s       bachelor's degree   F       42242.738000
                              M       36808.968000
          secondary education F       23521.394364
                              M       27733.323200

In [50]:
# Getting our income median values:

display("INCOME MEDIAN")
income_median = non_null_df.pivot_table(index=['age_group', 'education', 'gender'], values='total_income', aggfunc='median')
income_median

'INCOME MEDIAN'

total_income
age_group education           gender              
10s       secondary education F         17954.8410
                              M         14817.5910
          some college        F         19686.4940
                              M         14575.7170
20s       bachelor's degree   F         23440.0090
                              M         30717.2190
          primary education   F         17781.3660
                              M         27933.8285
          secondary education F         18247.9170
                              M         24664.3550
          some college        F         19925.4750
                              M         26768.9370
30s       bachelor's degree   F         26205.4365
                              M         33907.3860
          graduate degree     F         17822.7570
                              M         18551.8460
          primary education   F         18962.6750
                              M         19984.7195
          secondary education F         20658.3640
                              M         26323.0580
          some college        F         25298.5690
                              M         34596.9060
40s       bachelor's degree   F         27631.7065
                              M         35307.1870
          graduate degree     M         31771.3210
          primary education   F         21215.4590
                              M         21887.8250
          secondary education F         21148.9775
                              M         26940.5820
          some college        F         29323.6730
                              M         30965.6450
50s       bachelor's degree   F         28836.4210
                              M         34446.4820
          graduate degree     M         42945.7940
          primary education   F         14408.9025
                              M         23361.5090
          secondary education F         21245.6610
                              M         24600.6035
          some college        F         25327.2270
                              M         29405.0380
60s       bachelor's degree   F         27213.5890
                              M         33527.2120
          primary education   F         26423.8530
                              M         19490.9385
          secondary education F         22469.9355
                              M         24445.6715
          some college        F         37146.5350
                              M         27938.4350
70s       bachelor's degree   F         42271.4370
                              M         36808.9680
          secondary education F         24525.2240
                              M         31943.8490

In this scenario, we will be using the median values, purely for the fact that the means and medians of each age group are varied which implies some outliers that are skewing our data.


In [51]:
# Filling in our missing income values grouped by age, education and gender with ther median values:

df['total_income'] = df.groupby(['age_group', 'education', 'gender'])['total_income'].transform(lambda x: x.fillna(x.median()))

In [52]:
# Checking if we got any errors:

df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,NaN,transactions with commercial real estate,70s


An entry with a missing value was found that did not correspond to our conditions (there is no 'total_income' data on the female '70s' age group who had 'primary education' as their highest level of education. We will fill in these missing value with the median of all '70s' age group females.

In [53]:
# Filling in our remaining missing income values grouped by age and gender with their median values:

df['total_income'] = df.groupby(['age_group', 'gender'])['total_income'].transform(lambda x: x.fillna(x.median()))

In [54]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


In [55]:
#Checking if our fixed 'total_income' column matches the DataFrame's total no. of entries

len(df) == len(df['total_income'])

True

###  Restoring values in `days_employed`

We will also be using conditions based off of groupings of age, education and gender as they each represent a highly likely employment timeline in and of itself.

In [56]:
# Finding mean values of 'days_employed' based on our conditions:

display("EXPERIENCE MEAN")
experience_mean = non_null_df.pivot_table(index=['age_group', 'education', 'gender'], values='days_employed', aggfunc='mean')

experience_mean

'EXPERIENCE MEAN'

days_employed
age_group education           gender               
10s       secondary education F          544.908728
                              M          565.972960
          some college        F          737.787063
                              M          885.268574
20s       bachelor's degree   F         1229.438046
                              M         1215.447321
          primary education   F          940.116446
                              M         1351.846917
          secondary education F         1278.931460
                              M         1176.658020
          some college        F         1051.380572
                              M         1042.549608
30s       bachelor's degree   F         2049.171276
                              M         1979.140383
          graduate degree     F         5968.075884
                              M         2351.431934
          primary education   F         1892.938206
                              M         1588.833962
          secondary education F         2104.423361
                              M         1995.137667
          some college        F         1569.746944
                              M         1591.830896
40s       bachelor's degree   F         2888.384212
                              M         2299.679499
          graduate degree     M          409.200149
          primary education   F         2188.765908
                              M         1461.718420
          secondary education F         2939.723164
                              M         2447.925536
          some college        F         2348.308122
                              M         2634.232985
50s       bachelor's degree   F         3548.060836
                              M         2849.059271
          graduate degree     M         5352.038180
          primary education   F         2631.590189
                              M         3912.835935
          secondary education F         3543.115813
                              M         2746.717804
          some college        F         2497.116204
                              M         2931.070130
60s       bachelor's degree   F         5030.748980
                              M         3285.266583
          primary education   F         4152.333798
                              M         2564.903700
          secondary education F         4272.005194
                              M         2551.867709
          some college        F         2698.966448
                              M         1527.560229
70s       bachelor's degree   F         8244.960352
                              M        13923.375090
          secondary education F         2825.527601
                              M         2959.423742

In [57]:
# Finding median values of 'days_employed' based on our conditions:

display("EXPERIENCE MEDIAN")
experience_median = non_null_df.pivot_table(index=['age_group', 'education', 'gender'], values='days_employed', aggfunc='median')

experience_median

'EXPERIENCE MEDIAN'

days_employed
age_group education           gender               
10s       secondary education F          640.290559
                              M          565.972960
          some college        F          746.476294
                              M          885.268574
20s       bachelor's degree   F         1025.549623
                              M          969.488472
          primary education   F         1030.219648
                              M         1059.805662
          secondary education F         1135.326998
                              M          946.198260
          some college        F          810.900602
                              M          992.939655
30s       bachelor's degree   F         1638.031786
                              M         1479.544374
          graduate degree     F         5968.075884
                              M         2351.431934
          primary education   F         1347.644931
                              M          951.096743
          secondary education F         1696.644338
                              M         1520.591173
          some college        F         1151.911922
                              M         1277.265112
40s       bachelor's degree   F         2214.824703
                              M         1729.600774
          graduate degree     M          409.200149
          primary education   F         1452.698328
                              M          927.789271
          secondary education F         2221.168228
                              M         1818.660700
          some college        F         1807.107221
                              M         2204.552255
50s       bachelor's degree   F         2704.640715
                              M         2080.740408
          graduate degree     M         5352.038180
          primary education   F         2398.693041
                              M         1938.133640
          secondary education F         2503.837397
                              M         1848.717053
          some college        F         1958.675765
                              M         1781.073877
60s       bachelor's degree   F         3506.868862
                              M         2540.545483
          primary education   F         4091.139438
                              M         2564.903700
          secondary education F         2983.924098
                              M         1725.280678
          some college        F         1498.732915
                              M          589.421058
70s       bachelor's degree   F         8615.516055
                              M        13923.375090
          secondary education F         1678.969771
                              M         1899.257071

Yet again, we will be using the median to fill in our missing 'days_employed' values as the large difference in means and medians show a presence of heavy outliers at each subset of our groups.

In [58]:
# Filling in our missing days_employed values grouped by age, education and gender with ther median values:

df['days_employed'] = df.groupby(['age_group', 'education', 'gender'])['days_employed'].transform(lambda x: x.fillna(x.median()))


In [59]:
# Checking if our function worked:

df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,24525.224,transactions with commercial real estate,70s


Yet again, we get the same entry whereby the missing values were not filled up due to an absence of data for 'primary education' . We will fill this up with the 'days_employed' median of the general female '70s' age group.

In [60]:
# Replacing missing values

df['days_employed'] = df.groupby(['age_group', 'gender'])['days_employed'].transform(lambda x: x.fillna(x.median()))

In [61]:
# Checking if the no. of entries in our fixed column tallies with overall entries:

len(df['days_employed']) == len(df)

True

In [62]:
# Check the entries in all columns - make sure we fixed all missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17887 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          17887 non-null  int64  
 1   days_employed     17887 non-null  float64
 2   dob_years         17887 non-null  int64  
 3   education         17887 non-null  object 
 4   education_id      17887 non-null  int64  
 5   family_status     17887 non-null  object 
 6   family_status_id  17887 non-null  int64  
 7   gender            17887 non-null  object 
 8   income_type       17887 non-null  object 
 9   debt              17887 non-null  int64  
 10  total_income      17887 non-null  float64
 11  purpose           17887 non-null  object 
 12  age_group         17887 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.9+ MB


## Categorization of data

**Categorizing categorial data**
<br>
<br>
Firstly, we need to categorize our 'purpose' column into distinct and readable groups. 
<br>
<br>Secondly, it's important to subset 'marital_status' into groups that have single and double income respectively, as this criteria will affect an individual's ability to support their children on top of getting a loan.

In [63]:
# Replacing long string in 'purpose' column into distinct categories: 

def purpose_fix(purpose):
    if 'wedding' in purpose:
        return 'wedding'
    if 'edu' in purpose:
        return 'education'
    if 'university' in purpose:
        return 'education'
    if 'hous' in purpose:
        return 'property'
    if 'prop' in purpose:
        return 'property'
    if 'real estate' in purpose:
        return 'property'
    elif 'car' in purpose:
        return 'car'
            
        
df['purpose'] = df['purpose'].apply(purpose_fix)

df['purpose'].value_counts()

property     9047
car          3558
education    3333
wedding      1949
Name: purpose, dtype: int64

In [64]:
# categorizing family status types by double income whereby 0 = no, 1 = yes:

def double_income(family):
    if family == 'married':
        return 1
    if family == 'civil partnership':
        return 1
    if family == 'unmarried':
        return 0
    if family == 'divorced':
        return 0
    elif family == 'widow / widower':
        return 0
    
df['double_income'] = df['family_status'].apply(double_income)


**Categorizing numerical data**
<br>
<br>
We need to group 'total_income' into income brackets as income is a bank's foremost determining factor when assessing an individual's credit worthiness. Income brackets will allows us to hold income constant while we look for other credit score variables.

In [65]:
# Getting a statistical summary of our 'total_income' column:
    
df['total_income'].describe()

count     17887.000000
mean      27418.410475
std       16074.437806
min        3418.824000
25%       18159.869500
50%       24100.176000
75%       32437.949500
max      362496.645000
Name: total_income, dtype: float64

As we do not have the country of origins of this is dataset, we cannot assume income brackets. Having said that, will use the 25th, 50th and 75th percentile values as the benchmark for our range in brackets; 1 = low, 2 = low-mid, 3 = mid-high, 4 = high. Income brackets/groups allows us eliminate income as a changing variable at the group level and dial in on our objectives of family status and no. of children.

In [66]:
# Creating function for categorizing into different numerical groups based on ranges

def income_grouping(income):
    if income <= 18159.8695:
        return 1
    if income < 24100.176:
        return 2
    if income < 32437.949500:
        return 3
    elif income >= 32437.949500:
        return 4

df['income_bracket'] = df['total_income'].apply(income_grouping)

In [67]:
# Checking the unique counts of our income brackets:

df['income_bracket'].value_counts().sort_index()


1    4472
2    4471
3    4472
4    4472
Name: income_bracket, dtype: int64

## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [68]:
# Calculating default-rate based on the number of children

df.groupby('children')['debt'].mean()

children
0    0.081521
1    0.094306
2    0.094815
3    0.080745
4    0.100000
5    0.000000
Name: debt, dtype: float64

In [69]:
# Calculating the default rate by children and income bracket:

df.groupby(['income_bracket', 'children']).agg({'debt':['mean', 'size']})

debt      
                             mean  size
income_bracket children                
1              0         0.088459  2634
               1         0.097414  1160
               2         0.095076   589
               3         0.100000    80
               4         0.142857     7
               5         0.000000     2
2              0         0.080258  2791
               1         0.107330  1146
               2         0.105960   453
               3         0.072464    69
               4         0.000000    11
               5         0.000000     1
3              0         0.089466  2772
               1         0.096429  1120
               2         0.096907   485
               3         0.112500    80
               4         0.153846    13
               5         0.000000     2
4              0         0.068124  2745
               1         0.075690  1123
               2         0.082329   498
               3         0.043011    93
               4         0.111111     9
               5         0.000000     4

**Conclusion**

We can see definitely see an uptrend of default rates from having (0) children to having (1) and (2) children respectively. Anything above those numbers have varying trends, but because the sample sizes are low, we will ignore them for now till we get more data on those individuals.
<br>
Hypothesis accepted.


**Is there a correlation between family status and paying back on time?**

In [70]:
# Calculating default-rate based on family status:

df.groupby('family_status')['debt'].mean()


family_status
civil partnership    0.099382
divorced             0.074262
married              0.079365
unmarried            0.105008
widow / widower      0.065817
Name: debt, dtype: float64

In [71]:
#Calculating default rate based on 'double_income' status:

df.groupby('double_income')['debt'].mean()

double_income
0    0.092621
1    0.084474
Name: debt, dtype: float64

**Conclusion**

We found a slight downward trend regarding default rates of individuals who did not have double income and those who did. 
<br>
Hypothesis accepted: individuals who are in partnerships that allow for double-income, are more likely to repay their loans than those with only one income source.

**Is there a correlation between income level and paying back on time?**

In [72]:
# Calculating default-rate based on income level

df.groupby('income_bracket')['debt'].mean()

income_bracket
1    0.091905
2    0.089465
3    0.092576
4    0.071109
Name: debt, dtype: float64

**Conclusion**

We cannot deduce much in income brackets, apart from the fact the highest income bracket defaults the least. Thus, we can conclude that in terms of repaying a loan, income levels affect the lower to mid income brackets more so than the very highest bracket.

**How does credit purpose affect the default rate?**

In [73]:
# Check the percentages for default rate for each credit purpose:

df.groupby('purpose')['debt'].mean()

purpose
car          0.099775
education    0.098110
property     0.077263
wedding      0.083120
Name: debt, dtype: float64

**Conclusion**

Our observations show the higest default rates are due to credit purposes of buying a car and education. We can infer from this that car and student loans are much shorter in duration, and hence have a larger sum to be paid every month, more so than a property loan that can stretch on for 30+ years on average. Property loans and wedding loans are also loan types whereby it isn't unusual to expect the loan to be shared amongst two participating individuals, therefore easing the burden on an individual.


# General Conclusion 

Data Overview:
<br>
<br>
At the start to our analysis on a bank's customer's credit worthiness, we found missing values in columns 'days_employed' and 'total_income'. These values were symmetrical amongst both columns and upon grouping them by 'income_type', found they had very random distributions across the dataset. It was decided that they weren't missing because of other factors and were purely missing at random.

Data Transformation:
<br>
1. Illogical values in the 'children' column were dropped as there was no way of making sense of it.
2. The 'days_employed' column had two strange unique values. One being negative values which was converted to its absolute value and the other being extremely overestimated numbers that did not make logical sense. The latter was grouped by their 'income_type' and it was apparent they fit a pattern of individuals who do not have a regular income stream ('retirees'). These rows were dropped as they hardly lend a hand in determining our overall project hypothesis of finding out credit worthiness of future bank customers. In other words, a bank wouldn't be giving out loans to retirees on a regular basis. 
3.  The 'education' column had plenty of unique duplicates that was rectified by merely converting all values to the lower case.

Missing Values:
1. We categorized our ages into age groups to better summarize the ages of our entries.
2. We filled in our missing values from both 'days_employed' and 'total_income' columns with their means based off grouping of age groups, gender and education levels.

Data Categorization:
1. The purpose column was categorized into distinct categorical groups to summarize their long string values.
2. Family_status was categorized into those who have double-income status and those who do not.
3. Total_income was categorized into income brackets to better understand its underlying effect on default rates when investigating an individual's family status and no. of children.

**Overall Conclusion**
<br>Our two major hypotheses are accepted. 

An increasing number of children from 0 > 1 > 2 reflected an increasing rate of credit default. Beyond that however, our sample size was too small to give an accurate picture. 
<br>
<br>
Similarly, family status when broken down into categories of double income and single income reflected a disparity between loan default rates. It was apparent that single income individuals were defaulting at a slightly higher rate than double income individuals.
<br>
<br>
Income levels unfortunately did not show any patterns as the income brackets used was a very rudimentary one. If we had knowledge on the country of origins of our dataset, we could more accurately predict income brackets and thus their role in credit default.